# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

In [10]:
from google.colab import files
uploaded = files.upload()

Saving HC ROMS.zip to HC ROMS.zip


In [11]:
uploaded = files.upload()

Saving ROMS.zip to ROMS.zip


In [12]:
!python -m atari_py.import_roms .

copying adventure.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Air Raid (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Alien.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying crazy_climber.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Crazy Climber.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/crazy_climber.bin
copying elevator_action.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Elevator Action (Prototype).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/elevator_action.bin
copying gravitar.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Gravitar.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_

Colab에서 Gym render 사용하기 위한 설치


In [4]:
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

In [5]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

In [7]:
from pyvirtualdisplay import Display
display = Display(visible=False, size=(400, 300)) 
display.start()

# 1. Test Random Environment with OpenAI Gym

In [33]:
import gym 
import random

In [34]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [35]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [36]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render(mode='rgb_array') #mode='rgb_array'로 해야 함
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:70.0
Episode:2 Score:120.0
Episode:3 Score:135.0
Episode:4 Score:55.0
Episode:5 Score:55.0


# 2. Create a Deep Learning Model with Keras

In [37]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [38]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [39]:
#del model

In [40]:
model = build_model(height, width, channels, actions)

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 6)                

# 3. Build Agent with Keras-RL

In [42]:
#pip install keras-rl

In [43]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [44]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, 
                   dueling_type='avg',
                   nb_actions=actions,
                   nb_steps_warmup=1000
                  )
    return dqn

In [45]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [46]:
model._make_predict_function()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


In [47]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
  603/10000: episode: 1, duration: 12.998s, episode steps: 603, steps per second:  46, episode reward: 120.000, mean reward:  0.199 [ 0.000, 30.000], mean action: 2.517 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1191/10000: episode: 2, duration: 141.683s, episode steps: 588, steps per second:   4, episode reward: 105.000, mean reward:  0.179 [ 0.000, 25.000], mean action: 2.276 [0.000, 5.000],  loss: 16.981854, mean_q: 5.768256, mean_eps: 0.901405
 1592/10000: episode: 3, duration: 283.432s, episode steps: 401, steps per second:   1, episode reward: 55.000, mean reward:  0.137 [ 0.000, 20.000], mean action: 2.446 [0.000, 5.000],  loss: 1.256998, mean_q: 4.732006, mean_eps: 0.874810
 2635/10000: episode: 4, duration: 738.621s, episode steps: 1043, steps per second:   1, episode reward: 145.000, mean reward:  0.139 [ 0.000, 30.000], mean action: 2.513 [0.000, 5.000],  loss: 0.841225, mean_q: 3.753233, mean_eps: 0.809830
 3262/10000: episode: 5, dura

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


C:\Users\samsung\anaconda3\lib\site-packages\gym\envs\atari\environment.py:255: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode 1: reward: 335.000, steps: 852
Episode 2: reward: 155.000, steps: 857
Episode 3: reward: 50.000, steps: 378
Episode 4: reward: 110.000, steps: 596
Episode 5: reward: 110.000, steps: 374
Episode 6: reward: 105.000, steps: 685
Episode 7: reward: 80.000, steps: 651
Episode 8: reward: 195.000, steps: 862
Episode 9: reward: 80.000, steps: 509
Episode 10: reward: 255.000, steps: 1331
147.5


-------------

# 4. Reloading Agent from Memory (agent 저장 과정이라 지금 당장 필요하지 않음)

In [ ]:
dqn.save_weights('./dqn_weights_01.h5f')

NameError: name 'dqn' is not defined

In [ ]:
#del model, dqn

In [ ]:
#dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')